In [ ]:
!pip install medmnist

In [ ]:
%pip install torcheval

In [ ]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.utils.data as data
import torchvision.transforms as transforms
import torch.nn.functional as F
from sklearn import metrics

import medmnist
from medmnist import INFO, Evaluator

In [ ]:
print(f"MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

In [ ]:
data_flag = 'breastmnist'
download = True

NUM_EPOCHS = 5
BATCH_SIZE = 32
RANDOM_SEARCH_TRIALS = 10
info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

print("CHANNELS:", n_channels)
print("CLASSES:", n_classes)

DataClass = getattr(medmnist, info['python_class'])
print(DataClass)

In [ ]:
def get_device():
    return "cuda" if torch.cuda.is_available() else "cpu"

device = get_device()
print(device)

## First, we read the MedMNIST data, preprocess them and encapsulate them into dataloader form.

In [ ]:
# preprocessing
data_transform = transforms.Compose([
   # transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

train_transforms = transforms.Compose([
                                # transforms.RandomRotation(30),
                                # transforms.RandomHorizontalFlip(),
                                transforms.Resize(256),
                                transforms.ToTensor(),
                                transforms.Normalize([0.5], [0.5])])

validation_transforms = transforms.Compose([
                                transforms.Resize(256),
                                transforms.ToTensor(),
                                transforms.Normalize([0.5],[0.5])])


test_transforms = transforms.Compose([
                                transforms.Resize(256),
                                transforms.ToTensor(),
                                transforms.Normalize([0.5],[0.5])])



# Load the original dataset
original_dataset = medmnist.dataset.BreastMNIST(split='train', transform=data_transform, download=download)

train_dataset = DataClass(split='train', transform=train_transforms, download=download)
validation_dataset = DataClass(split='val', transform=validation_transforms, download=download)
test_dataset = DataClass(split='test', transform=test_transforms, download=download)
pil_dataset = DataClass(split='train', download=download)

# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = data.DataLoader(dataset=validation_dataset, batch_size=BATCH_SIZE, shuffle=False)
train_loader_at_eval = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
print(train_dataset)
print("===================")
print(test_dataset)

In [ ]:
# montage

original_dataset.montage(length=20)

## Then, we define a simple model for illustration, object function and optimizer that we use to classify.

In [ ]:
import torch
import torchvision.models as models
import torch.nn as nn

# resnet18 = models.resnet18(pretrained=True)

class ExtendedNetwork(nn.Module):
    def __init__(self, n_classes=2):
        # super(ExtendedNetwork, self).__init__()
        super().__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False)

        # Freeze all layers of the original ResNet18 model
        for param in self.resnet.parameters():
            param.requires_grad = False
        num_features = self.resnet.fc.in_features
        # self.resnet.fc = nn.Linear(num_features, 512)


        self.resnet.fc = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.BatchNorm1d(512),  # Match the batch norm to the output features of ResNet
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, n_classes),
        )

    def forward(self, x):
        x = self.resnet(x)    # Pass input through ResNet18
        return x

In [ ]:
network = ExtendedNetwork()
network.to(device=device)

print(network)

In [ ]:
def accuracy(outputs, targets):
    pred = outputs.argmax(dim=1, keepdim=True)
    correct = pred.eq(targets.view_as(pred)).sum().item()
    return correct / len(targets)

## Next, we can start to train and evaluate!

In [ ]:
# !pip install optuna "ray[tune]"

In [ ]:
from itertools import product

LEARNING_RATES = [0.0001, 0.00015, 0.001, 0.0015]
MOMENTUM_VALUES = [0.9, 0.92, 0.93, 0.95]
WEIGHT_DECAYS = [1e-4, 15e-4, 17e-4, 1e-3]
optimizers = [optim.Adagrad, optim.Adam, optim.AdamW, optim.SGD]
patiences = [5, 7]
schedulers = [torch.optim.lr_scheduler.ReduceLROnPlateau, torch.optim.lr_scheduler.StepLR]
loss_functions = [nn.NLLLoss(), nn.CrossEntropyLoss()]

hyperparameters_grid = product(LEARNING_RATES, MOMENTUM_VALUES, WEIGHT_DECAYS, optimizers, patiences, schedulers, loss_functions)

In [ ]:
def get_optimizer(optimizer_class, parameters, lr, momentum, weight_decay):
    # Check which optimizer is being used and adjust parameters accordingly
    if optimizer_class in [optim.SGD]:
        # SGD uses both momentum and weight_decay
        return optimizer_class(parameters, lr=lr, momentum=momentum, weight_decay=weight_decay)
    elif optimizer_class in [optim.Adam, optim.AdamW, optim.Adagrad]:
        # Adam and AdamW do not use momentum
        return optimizer_class(parameters, lr=lr, weight_decay=weight_decay)
    else:
        # Default fallback for any other optimizers
        raise ValueError(f"Unsupported optimizer class {optimizer_class}")

In [ ]:
def get_scheduler(scheduler_class, optimizer_class, patience=None):
    # Check the type of scheduler to initialize it correctly
    if scheduler_class == torch.optim.lr_scheduler.ReduceLROnPlateau:
        # ReduceLROnPlateau requires 'patience' and benefits from a 'factor'
        return scheduler_class(optimizer_class, mode='min', patience=patience, factor=0.1, verbose=True)
    elif scheduler_class == torch.optim.lr_scheduler.StepLR:
        # StepLR does not use 'patience' but uses 'step_size' and 'gamma'
        return scheduler_class(optimizer_class, step_size=50, gamma=0.1)
    else:
        # For other schedulers, adjust as necessary or raise an error
        raise ValueError("Unsupported scheduler class provided")

In [ ]:
def test(split):
    # Load the model with the best accuracy
    network.load_state_dict(torch.load('best_model.pth'))
    network.eval()
    y_true = torch.tensor([], device=device)
    y_score = torch.tensor([], device=device)
    collected_inputs = torch.tensor([], device=device)
    collected_targets = torch.tensor([], device=device)

    data_loader = train_loader_at_eval if split == 'train' else test_loader

    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            
            outputs = network(inputs)

            if task == 'multi-label, binary-class':
                targets = targets.to(torch.float32)
                outputs = outputs.softmax(dim=-1)
            else:
                targets = targets.squeeze().long()
                outputs = outputs.softmax(dim=-1)
                targets = targets.float().resize_(len(targets), 1).squeeze()

            y_true = torch.cat((y_true, targets), 0)
            y_score = torch.cat((y_score, outputs), 0)
            collected_inputs = torch.cat((collected_inputs, inputs), 0)
            collected_targets = torch.cat((collected_targets, targets), 0)

        y_true = y_true.cpu().numpy()
        y_score = y_score.cpu().detach().numpy()
        collected_inputs = collected_inputs.cpu()
        collected_targets = collected_targets.cpu()

        evaluator = Evaluator(data_flag, split)
        metrics = evaluator.evaluate(y_score)

        print('%s  auc: %.3f  acc:%.3f' % (split, *metrics))

        return metrics, collected_inputs, collected_targets, y_true, y_score

# print('==> Evaluating ...')
# train_inputs, train_targets, train_true, train_scores = test('train')
# test_inputs, test_targets, test_true, test_scores = test('test')


In [ ]:
import random
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # Sets the seed for all GPUs
random.seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from itertools import product
from tqdm import tqdm
import pandas as pd
from torch.nn.functional import log_softmax
import logging

# Hyperparameters
# LEARNING_RATES = [0.0001, 0.001, 0.01]
# MOMENTUM_VALUES = [0.9, 0.92, 0.93]
# WEIGHT_DECAYS = [0.0001, 0.001, 0.01]
PATIENCE = 7  # Patience for early stopping

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
network.to(device)

# Prepare grid search
hyperparameters_grid = product(LEARNING_RATES, MOMENTUM_VALUES, WEIGHT_DECAYS, optimizers, patiences, schedulers, loss_functions)
combination_number = len(LEARNING_RATES) * len(MOMENTUM_VALUES) * len(WEIGHT_DECAYS) * len(optimizers) * len(patiences) * len(schedulers) * len(loss_functions)


results_df = pd.DataFrame(columns=['Trial', 'AUC', 'Accuracy', 'Learning Rate', 'Momentum', 'Weight Decay', 'Optimizer', 'Patience', 'Scheduler', 'Loss Function'])

# Best model tracking
best_val_accuracy = 0
best_train_accuracy = 0
best_hyperparameters = {}


# Lists to store average accuracy
avg_train_accuracies = []
avg_val_accuracies = []

# Lists to store average losses
avg_train_losses = []
avg_val_losses = []


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Training and validation
for trial, params in enumerate(hyperparameters_grid):
    try:
        print(f"\nTrial {trial + 1}/{combination_number}")
        learning_rates, momentum, weight_decay, optimizer_class, patience, scheduler_class, loss_function = params
        # learning_rate = np.random.choice(LEARNING_RATES)
        # momentum = np.random.choice(MOMENTUM_VALUES)
        # weight_decay = np.random.choice(WEIGHT_DECAYS)


        print(f"Testing on Hyperparameters: Learning Rate: {learning_rates}, Momentum: {momentum}, Weight Decay: {weight_decay}, Optimizer: {optimizer_class}, Patience: {patience}, Scheduler: {scheduler_class}, Loss Function: {loss_function}")

        # # Optimizer setup
        # print(f"Optimizer class type: {optimizer_class}")
        # print(f"Optimizer class type: {type(optimizer_class)}")

        # optimizer = optim.Adagrad(network.parameters(), lr=learning_rates, weight_decay=weight_decay, maximize=False)
        optimizer = get_optimizer(optimizer_class, network.parameters(), learning_rates, momentum, weight_decay)
        # scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.2, verbose=True)
        scheduler = get_scheduler(scheduler_class, optimizer, patience)

        early_stopping_counter = 0
        best_epoch_val_loss = float('inf')

        for epoch in range(NUM_EPOCHS):
            # Training loop
            network.train()
            running_loss = 0.0
            running_accuracy = 0.0

            for inputs, targets in tqdm(train_loader, desc="(Train) Epoch: " + str(epoch) + " on " + str(device)):
                # forward + backward + optimize
                inputs, targets = inputs.to(device), targets.to(device)

                optimizer.zero_grad()
                outputs = network(inputs)

                targets = targets.squeeze().long()
                
                if loss_function == nn.NLLLoss():
                    outputs = log_softmax(outputs, dim=1)
                # Call Loss function
                    loss = loss_function(outputs, targets)
                else:
                    loss = loss_function(outputs, targets)
                
                # Call backward function
                loss.backward()
                optimizer.step()

                running_loss += loss.item() #/ len(inputs)
                running_accuracy += accuracy(outputs.exp(), targets)

            # Train Accuracy calculation
            epoch_accuracy = running_accuracy / len(train_loader)
            avg_train_accuracies.append(epoch_accuracy)
            
            # Train Loss calculation
            epoch_loss = running_loss / len(train_loader)
            avg_train_losses.append(epoch_loss)

            print("Train - Loss: {:.6f}, Accuracy: {:.2f}%".format(epoch_loss, running_accuracy / len(train_loader) * 100), "\n")

            # Validation
            network.eval()
            val_running_loss = 0.0
            val_running_accuracy = 0.0
            with torch.no_grad():
                for val_inputs, val_targets in tqdm(validation_loader, desc="(Validation) Epoch: " + str(epoch) + " on " + str(device)):
                    val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)

                    val_outputs = network(val_inputs)

                    # val_outputs = F.softmax(val_outputs, dim=1)

                    val_targets = val_targets.squeeze().long()
                    if loss_function == nn.NLLLoss():
                        val_outputs = F.log_softmax(val_outputs, dim=1)
                        loss = F.nll_loss(val_outputs, val_targets)
                        # Call Backward function
                        loss.backward()
                    else:
                        loss = loss_function(val_outputs, val_targets)

                    val_running_loss += loss.item()
                    val_running_accuracy += accuracy(val_outputs, val_targets)

            # Validation Accuracy calculation
            val_epoch_accuracy = val_running_accuracy / len(validation_loader)
            avg_val_accuracies.append(val_epoch_accuracy)

            # Validation Loss calculation
            val_epoch_loss = val_running_loss / len(validation_loader)
            avg_val_losses.append(val_epoch_loss)

            scheduler.step(val_epoch_loss)

            # Early stopping based on validation loss
            if val_epoch_loss < best_epoch_val_loss:
                best_epoch_val_loss = val_epoch_loss

            print("Validation - Loss: {:.6f}, Accuracy: {:.2f}%".format(val_epoch_loss, val_running_accuracy / len(validation_loader) * 100), "\n")

            # Check if this trial has the best accuracy
            if running_accuracy > best_train_accuracy:
                best_train_accuracy = running_accuracy

            if val_running_accuracy > best_val_accuracy:
                best_val_accuracy = val_running_accuracy
                best_hyperparameters = {
                    "learning_rate": learning_rates,
                    "momentum": momentum,
                    "weight_decay": weight_decay,
                    "optimizer": optimizer_class,
                    "patience": patience,
                    "scheduler": scheduler_class,
                    "loss_function": loss_function
                }

                model_save_path = f"best_model_seed_{42}.pth"
                torch.save(network.state_dict(), "best_model.pth")
                print("Saved the new best model with seed 42 and validation accuracy of", best_val_accuracy)  

                            # Evaluate the model on test data
                test_metrics, _, _, _, _ = test('test')  # Ensure that you have a test_loader defined
                test_auc, test_accuracy = test_metrics
            
                # Create a DataFrame for the new row
                new_row = pd.DataFrame({
                    'Trial': [trial + 1],
                    'AUC': [test_auc],
                    'Accuracy': [test_accuracy],
                    'Learning Rate': [learning_rates],
                    'Momentum': [momentum],
                    'Weight Decay': [weight_decay],
                    'Optimizer': [optimizer_class],
                    'Patience': [patience],
                    'Scheduler': [schedulers],
                    'Loss Function': [loss_function]
                })
                results_df = pd.concat([results_df, new_row], ignore_index=True)
                results_df.to_csv("check_results.csv", index=False)
                print("Saved results to CSV for checking.")   
        
        logging.info(f"Finished trial {trial+1}")
    
    except Exception as e:
        logging.error(f"Error in trial {trial+1} with params: {params}. Error: {str(e)}")
        continue
# Print the best hyperparameters and accuracy
print("\nBest Hyperparameters:")
print(best_hyperparameters)
print("Best Training Accuracy: {:.2f}%".format(best_train_accuracy / len(train_loader) * 100))
print("Best Validation Accuracy: {:.2f}%".format(best_val_accuracy / len(validation_loader) * 100))


In [ ]:
# from matplotlib import pyplot as plt

# # Print the best hyperparameters and accuracy
# print("\nBest Hyperparameters:")
# print(best_hyperparameters)
# print("Best Training Accuracy: {:.2f}%".format(best_train_accuracy / len(train_loader) * 100))
# print("Best Validation Accuracy: {:.2f}%".format(best_val_accuracy / len(validation_loader) * 100))

# # Plotting the training and validation loss
# plt.figure(figsize=(10, 5))
# plt.title("Training and Validation Loss")
# plt.plot(avg_train_losses, label="Training Loss", color="blue")
# plt.plot(avg_val_losses, label="Validation Loss", color="red")
# plt.xlabel("Epochs")
# plt.ylabel("Loss")
# plt.legend()
# plt.show()

# # Plotting the training and validation accuracies
# plt.figure(figsize=(10, 5))
# plt.title("Training and Validation Accuracy")
# plt.plot([100 * acc for acc in avg_train_accuracies], label="Training Accuracy", color="blue")
# plt.plot([100 * acc for acc in avg_val_accuracies], label="Validation Accuracy", color="red")
# plt.xlabel("Epochs")
# plt.ylabel("Accuracy (%)")
# plt.legend()
# plt.ylim(0, 100)  # Set y-axis to show percentages from 0 to 100
# plt.show()

In [ ]:
# evaluation

def test(split):
    network.eval()
    y_true = torch.tensor([], device=device)
    y_score = torch.tensor([], device=device)

    data_loader = train_loader_at_eval if split == 'train' else test_loader

    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = network(inputs)

            if task == 'multi-label, binary-class':
                targets = targets.to(torch.float32)
                outputs = outputs.softmax(dim=-1)
            else:
                targets = targets.squeeze().long()
                outputs = outputs.softmax(dim=-1)
                targets = targets.float().resize_(len(targets), 1)

            y_true = torch.cat((y_true, targets), 0)
            y_score = torch.cat((y_score, outputs), 0)

        y_true = y_true.cpu().numpy()
        y_score = y_score.cpu().detach().numpy()

        evaluator = Evaluator(data_flag, split)
        metrics = evaluator.evaluate(y_score)

        #print('%s  auc: %.3f  acc:%.3f' % (split, *metrics))
    return (split, *metrics), y_true, y_score


print('==> Evaluating ...')
# test('train')
# test('test')
train_result, _, _ = test("train")  # Unpack only the first returned value
print('%s  auc: %.3f  acc:%.3f' % train_result)
test_result, _, _ = test("test")  # Unpack only the first returned value
print('%s  auc: %.3f  acc:%.3f' % test_result)

## Deliverable 2

In [ ]:
%pip install --upgrade torcheval

## Calculating AUPRC

In [ ]:
from torcheval.metrics import MulticlassAUPRC
import torch

_, train_true, train_scores = test('train')
_, test_true, test_scores = test('test')

train_scores = torch.tensor(train_scores, device='cuda' if torch.cuda.is_available() else 'cpu')
train_true = torch.tensor(train_true, device='cuda' if torch.cuda.is_available() else 'cpu').long().squeeze()  # Ensure labels are long type
test_scores = torch.tensor(test_scores, device='cuda' if torch.cuda.is_available() else 'cpu')
test_true = torch.tensor(test_true, device='cuda' if torch.cuda.is_available() else 'cpu').long().squeeze()
# Assuming num_classes is the actual number of classes
num_classes = 2# len(torch.unique(torch.tensor(train_true)))  # Update based on your labels

# Initialize the Multiclass AUPRC metric
metric_train = MulticlassAUPRC(num_classes=n_classes)
metric_train.update(train_scores, train_true)

metric_test = MulticlassAUPRC(num_classes=n_classes)
metric_test.update(test_scores, test_true)

# Compute the final AUPRC
train_auprc_result = metric_train.compute()
test_auprc_result = metric_test.compute()

print("Train AUPRC result:", train_auprc_result)
print("Test AUPRC result:", test_auprc_result)

## Calculating Precision, Recall & F1 Scores

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

train_true = train_true.cpu()
train_scores = train_scores.cpu()
test_true = test_true.cpu()
test_scores = test_scores.cpu()

train_precision = precision_score(train_true, train_scores.argmax(dim=1), average='weighted')
test_precision = precision_score(test_true, test_scores.argmax(dim=1), average='weighted')

print("Train Precision Score:", train_precision)
print("Test Precision Score:", test_precision)

print()

train_recall = recall_score(train_true, train_scores.argmax(dim=1), average='weighted')
test_recall = recall_score(test_true, test_scores.argmax(dim=1), average='weighted')

print("Train Recall Score:", train_recall)
print("Test Recall Score:", test_recall)

print() 

train_f1 = f1_score(train_true, train_scores.argmax(dim=1), average='weighted')
test_f1 = f1_score(test_true, test_scores.argmax(dim=1), average='weighted')

print("Train F1 Score:", train_f1)
print("Test F1 Score:", test_f1)

## Deliverable 3

In [ ]:
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

train_fpr, train_tpr, train_thresholds = metrics.roc_curve(train_true, train_scores[:, 1])
train_roc_auc = metrics.auc(train_fpr, train_tpr)

test_fpr, test_tpr, test_thresholds = metrics.roc_curve(test_true, test_scores[:, 1])
test_roc_auc = metrics.auc(test_fpr, test_tpr)

plt.figure()
plt.plot(train_fpr, train_tpr, label=f'Train ROC curve (area = {train_roc_auc:.2f})')
plt.plot(test_fpr, test_tpr, label=f'Test ROC curve (area = {test_roc_auc:.2f})')
plt.plot([0, 1], [0, 1], "b--")  # Dashed diagonal
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
print(train_true.shape)
print(train_scores.shape)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

confusionMatrix = confusion_matrix(train_true, train_scores)
labels = ['Malignant', 'Benign'] # CHECK LABELS
display_confusion_matrix = ConfusionMatrixDisplay(confusion_matrix=confusionMatrix, display_labels=labels)
display_confusion_matrix.plot(cmap="Blues")
plt.show()

## Estimation Testing

In [ ]:
import torch
from PIL import Image
from torchvision import transforms

# Assume 'model' is your trained model
network.eval()

# Assuming you have a DataLoader for your test data
# For a specific image, you might load it directly as shown below
image, label = pil_dataset[20]

# Transform the image according to your model's expected input
# This should match your training transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Example size, adjust as necessary
    transforms.ToTensor(),
    # Include normalization if used during training
])

# Apply transformation
image = transform(image).unsqueeze(0)  # Add batch dimension

# Move image to the same device as your model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
image = image.to(device)
model = network.to(device)

with torch.no_grad():
    output = model(image)

# Assuming the output is logits; apply softmax for probabilities
probabilities = torch.softmax(output, dim=1).cpu().numpy().flatten()

# Assuming class 0 is 'benign' and class 1 is 'malignant'
classes = ['benign', 'malignant']
predicted_class = classes[probabilities.argmax()]
confidence = probabilities.max()

print(f"Predicted class: {predicted_class} with confidence {confidence:.2f}")


image_np = image.squeeze().cpu().numpy()

# If the image is grayscale (C, H, W) where C = 1, we convert it to (H, W) for matplotlib
if image_np.shape[0] == 1:  # Grayscale image, single channel
    image_np = image_np.squeeze(0)  # Now shape is (H, W)
elif image_np.shape[0] == 3:  # If it's a 3-channel image
    # Convert from (C, H, W) to (H, W, C) for RGB images
    image_np = np.transpose(image_np, (1, 2, 0))

# Display the image
plt.imshow(image_np, cmap='gray' if image_np.ndim == 2 else None)
plt.axis('off')
plt.show()